<a href="https://colab.research.google.com/github/GNAIXOAH/COMP562-Final/blob/main/562final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
import pandas as pd
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import StandardScaler
from sklearn.metrics.pairwise import cosine_similarity
import random


In [8]:
random.seed(562)
np.random.seed(562)

In [9]:
data_raw = pd.read_csv("/content/final_final.csv")
print(data_raw['Mode'].unique())

['Minor' 'Major']


In [10]:
data_onehot = data_raw.copy()
data_onehot['Mode'] = data_onehot['Mode'].apply(lambda x: 1 if x == "Major" else 0)
data_onehot = data_onehot.drop("Lyrics", axis=1)
print(data_onehot.columns)

Index(['Unnamed: 0', 'Title', 'Artist', 'Genre', 'Acousticness',
       'Danceability', 'Energy', 'Instrumentalness', 'Liveness', 'Loudness',
       'Speechiness', 'Valence', 'Tempo', 'Mode', 'Time Signature', 'Key',
       'Duration', 'Release Year', 'Year'],
      dtype='object')


In [11]:
data_full = data_onehot[['Title', 'Artist', 'Genre', 'Acousticness', 'Tempo', 'Time Signature', 'Key', 'Mode']]
data_full = data_full.dropna()
scaler = StandardScaler()
data_full.iloc[:, 3:] = scaler.fit_transform(data_full.iloc[:, 3:])


<ipython-input-11-ee9db767aa2b>:4: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[0.06507013 0.06507013 0.06507013 ... 0.06507013 0.06507013 0.06507013]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  data_full.iloc[:, 3:] = scaler.fit_transform(data_full.iloc[:, 3:])
<ipython-input-11-ee9db767aa2b>:4: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[-1.19995886 -1.47952101  0.47741404 ...  1.31610049  1.31610049
 -1.47952101]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  data_full.iloc[:, 3:] = scaler.fit_transform(data_full.iloc[:, 3:])
<ipython-input-11-ee9db767aa2b>:4: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[-1.22521403  0.81618393  0.81618393 ...  0.81618393  0.81618393
  0.81618

In [12]:


train_indices = np.random.choice(len(data_full), size=int(0.8 * len(data_full)), replace=False)
training_data = data_full.iloc[train_indices]
testing_data = data_full.drop(data_full.index[train_indices]) # Use data_full.index to get the actual index labels


In [13]:
testing_data.to_csv("testing_data.csv", index=False)


In [14]:
normalized_train = training_data.iloc[:, 3:].values
normalized_test = testing_data.iloc[:, 3:].values



In [15]:
model = Sequential([
    Dense(64, activation='relu', input_shape=(normalized_train.shape[1],)),
    Dense(32, activation='relu'),
    Dense(16, activation='relu'),
    Dense(8, activation='linear'),
    Dense(5, activation='linear')  # Changed output layer to 5 neurons
])

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [16]:
model.summary()


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense (Dense)                        │ (None, 64)                  │             384 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 32)                  │           2,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 16)                  │             528 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 8)                   │             136 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ (None, 5)                   │              45 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 3,173 (12.39 KB)

 Trainable params: 3,173 (12.39 KB)

 Non-trainable params: 0 (0.00 B)

In [17]:
model.compile(loss='mse', optimizer=Adam())


In [18]:
history = model.fit(normalized_train, normalized_train, epochs=1000, batch_size=32, validation_split=0.2, verbose=1)


Epoch 1/1000
22/22 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step - loss: 1.0668 - val_loss: 0.6809
Epoch 2/1000
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.7044 - val_loss: 0.4890
Epoch 3/1000
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.5144 - val_loss: 0.2986
Epoch 4/1000
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.2986 - val_loss: 0.1833
Epoch 5/1000
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.2531 - val_loss: 0.0993
Epoch 6/1000
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.1125 - val_loss: 0.0482
Epoch 7/1000
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0580 - val_loss: 0.0209
Epoch 8/1000
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0203 - val_loss: 0.0136
Epoch 9/1000
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0140 - val_loss: 0.0091
Epoch 10/1000
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0096 - val_loss: 0.0074
Epoch 11/1000
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0073 - val_loss: 0.0063
Epoch 12/1000
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/ste

In [19]:
encoder_model = Sequential(model.layers[:4])

In [20]:
latent_space = encoder_model.predict(normalized_test)

7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step


In [21]:
def matches_any_genre(genre_string, genres):
    return any([genre.lower() in genre_string.lower() for genre in genres])

In [23]:
songs_rnb_metal_soul = testing_data[testing_data['Genre'].apply(lambda x: matches_any_genre(x, ["R&B", "alternative metal", "soul"]))]
songs_pop_edm = testing_data[testing_data['Genre'].apply(lambda x: matches_any_genre(x, ["dance pop", "EDM"]))]
print({
    'Songs_RnB_Metal_Soul': songs_rnb_metal_soul,
    'Songs_Pop_EDM': songs_pop_edm
})

num_songs = 30
UserA = songs_rnb_metal_soul.sample(n=num_songs)
UserB = songs_rnb_metal_soul.sample(n=num_songs)
UserC = songs_pop_edm.sample(n=num_songs)
UserD = songs_pop_edm.sample(n=num_songs)
print({
    'User_A': UserA,
    'User_B': UserB,
    'User_C': UserC,
    'User_D': UserD
})

def calculate_overlap(user1, user2):
    return len(set(user1['Title']) & set(user2['Title']))

users = {'UserA': UserA, 'UserB': UserB, 'UserC': UserC, 'UserD': UserD}
results = np.zeros((len(users), len(users)))
for i, user1 in enumerate(users.values()):
    for j, user2 in enumerate(users.values()):
        if i == j:
            results[i, j] = len(user1)
        else:
            overlap_count = calculate_overlap(user1, user2)
            results[i, j] = overlap_count
print(results)

user_images = {name: encoder_model.predict(user.iloc[:, 3:].values).mean(axis=0) for name, user in users.items()}
results = np.zeros((len(user_images), len(user_images)))
for i, img1 in enumerate(user_images.values()):
    for j, img2 in enumerate(user_images.values()):
        if i == j:
            results[i, j] = 1
        else:
            sim = cosine_similarity([img1], [img2])[0][0]
            results[i, j] = sim
print(results)


def recommend_top_unheard(main_playlist, main_image, alt_playlist, n_songs):
    # Predict latent vectors for main and alt playlists
    main_playlist_latent = encoder_model.predict(main_playlist.iloc[:, 3:].values.astype(np.float32))
    alt_playlist_latent = encoder_model.predict(alt_playlist.iloc[:, 3:].values.astype(np.float32))

    # Create a DataFrame with latent space for alt_playlist
    unheard_playlist_latent = pd.DataFrame(
        alt_playlist_latent,
        columns=[f'latent_{i}' for i in range(alt_playlist_latent.shape[1])],
        index=range(len(alt_playlist))
    )
    unheard_playlist_latent['Title'] = alt_playlist['Title'].values

    # Initialize a 'Score' column with NaN
    unheard_playlist_latent['Score'] = np.nan

    # Calculate similarity scores for each song in the alt playlist vs the main_image
    for i, row in main_playlist.iterrows():
        # Get latent vector for the current song from main_playlist
        song_latent = encoder_model.predict(np.array([row.iloc[3:].values.astype(np.float32)]))[0]

        # Compute similarity between this song's latent vector and the main_image
        similarity_score = cosine_similarity([song_latent], [main_image])[0][0]

        # If the song appears in the alt_playlist, update its score
        # However, the logic here may need to be clarified. If we only want unheard songs,
        # we should only assign scores to songs not in main_playlist.
        # The current code assigns a score to songs in the main playlist if they appear in alt_playlist.
        # Typically, you'd only assign scores to unheard songs. Let's assume every song in alt_playlist
        # can be scored against the main_image, and we want the top n_songs from that.

        # The loop is currently iterating over songs in the main_playlist but using their scores
        # to set values in unheard_playlist_latent. This may not be the intended logic. Usually,
        # you'd just compute the similarity of all alt_playlist songs to main_image and pick top.
        # Let's revise the logic to compute all scores directly:

    # Instead of looping through main_playlist songs, just compute scores for all alt_playlist songs
    # against main_image directly:
    alt_vectors = alt_playlist_latent
    scores = cosine_similarity(alt_vectors, main_image.reshape(1, -1)) # main_image is a vector; reshape for similarity
    # scores will be a matrix of shape (len(alt_playlist), 1)
    # Assign these scores directly
    unheard_playlist_latent['Score'] = scores.ravel()

    # Filter out songs that main_playlist already heard (if intended)
    heard_titles = set(main_playlist['Title'])
    unheard_playlist_latent = unheard_playlist_latent[~unheard_playlist_latent['Title'].isin(heard_titles)]

    # Now get the top songs by score
    top_songs = unheard_playlist_latent.nlargest(n_songs, 'Score')

    # Merge top_songs with alt_playlist to get details (Artist, Genre, etc.)
    top_songs_info = pd.merge(
        alt_playlist[['Title','Artist','Genre']],
        top_songs[['Title','Score']],
        on='Title', how='right'
    )

    return top_songs_info


BtoA = recommend_top_unheard(UserA, user_images['UserA'], UserB, 5)
DtoA = recommend_top_unheard(UserA, user_images['UserA'], UserD, 5)
CtoA = recommend_top_unheard(UserA, user_images['UserA'], UserC, 5)
print(BtoA, DtoA, CtoA)

# 从整个数据库中推荐未听过的歌曲
from_database = recommend_top_unheard(UserA, user_images['UserA'], testing_data, 10)
print(from_database)

{'Songs_RnB_Metal_Soul':                                                  Title  \
1                                 All The Small Things   
25                                    Jumpin', Jumpin'   
41                                 With Arms Wide Open   
45      U Know What's Up (feat. Lisa "Left Eye" Lopes)   
71                                    Where I Wanna Be   
82                                           Wonderful   
85                                   Let's Get Married   
99                                 Don't Think I'm Not   
101                                           Survivor   
110                                  How You Remind Me   
140                                        Differences   
188                                      It's Over Now   
191                        Stutter - Double Take Remix   
197                                           AM To PM   
207                                         In the End   
235                                            

In [24]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
